# Benchmarking the Keplerian Solver

`simppler` uses a custom Keplerian solver written with [Numba](https://numba.readthedocs.io/en/stable/index.html). In this notebook, we do a quick benchmark comparison with [RadVel](https://radvel.readthedocs.io/).

We first copy the `profile()` function from RadVel's `kepler` module, and then add a block to test `simppler`'s solver.

In [ ]:
def profile():
    # Profile and compare C-based Kepler solver with
    # Python/Numpy implementation

    import timeit
    from radvel.kepler import _check_cext

    ecc = 0.1
    numloops = 5000
    print("\nECCENTRICITY = {}".format(ecc))

    for size in [10, 30, 100, 300, 1000]:

        setup = """\
from radvel.kepler import rv_drive
from simppler.kepler import rv_drive as simppler_rv_drive
import numpy as np
gc.enable()
ecc = %f
orbel = [32.468, 2456000.0, ecc, np.pi/2, 10.0]
t = np.linspace(2455000, 2457000, %d)
simppler_rv_drive(t, orbel)
""" % (ecc, size)

        if _check_cext():
            print("\nProfiling pure C code for an RV time series with {} "
                  "observations".format(size))
            tc = timeit.timeit('rv_drive(t, orbel, use_c_kepler_solver=True)',
                               setup=setup, number=numloops)
            print("Ran %d model calculations in %5.3f seconds" % (numloops, tc))
        else:
            print("\nC extension not available; skipping C profiling for {} observations".format(size))
            tc = None

        print("Profiling Python code for an RV time series with {} "
              "observations".format(size))
        tp = timeit.timeit('rv_drive(t, orbel, use_c_kepler_solver=False)',
                           setup=setup, number=numloops)
        print("Ran %d model calculations in %5.3f seconds" % (numloops, tp))
        if tc is not None and tc > 0:
            print("The C version runs %5.2f times faster" % (tp/tc))

        print("Profiling simppler code for an RV time series with {} "
              "observations".format(size))
        tp = timeit.timeit('simppler_rv_drive(t, orbel)',
                           setup=setup, number=numloops)
        print("Ran %d model calculations in %5.3f seconds" % (numloops, tp))
        if tc is not None and tc > 0:
            print("The C version runs %5.2f times faster" % (tp/tc))

    ecc = 0.7
    numloops = 5000
    print("\nECCENTRICITY = {}".format(ecc))

    for size in [30]:
            setup = """\
from radvel.kepler import rv_drive
from simppler.kepler import rv_drive as simppler_rv_drive
import numpy as np
gc.enable()
ecc = %f
orbel = [32.468, 2456000.0, ecc, np.pi/2, 10.0]
t = np.linspace(2455000, 2457000, %d)
simppler_rv_drive(t, orbel)
    """ % (ecc, size)

            if _check_cext():
                print("\nProfiling pure C code for an RV time series with {} "
                      "observations".format(size))
                tc = timeit.timeit('rv_drive(t, orbel, use_c_kepler_solver=True)',
                                   setup=setup, number=numloops)
                print("Ran %d model calculations in %5.3f seconds" % (numloops, tc))
            else:
                print("\nC extension not available; skipping C profiling for {} observations".format(size))
                tc = None

            print("Profiling Python code for an RV time series with {} "
                  "observations".format(size))
            tp = timeit.timeit('rv_drive(t, orbel, use_c_kepler_solver=False)',
                               setup=setup, number=numloops)
            print("Ran %d model calculations in %5.3f seconds" % (numloops, tp))
            if tc is not None and tc > 0:
                print("The C version runs %5.2f times faster" % (tp / tc))

            print("Profiling simppler code for an RV time series with {} "
                  "observations".format(size))
            tp = timeit.timeit('simppler_rv_drive(t, orbel)',
                               setup=setup, number=numloops)
            print("Ran %d model calculations in %5.3f seconds" % (numloops, tp))
            if tc is not None and tc > 0:
                print("The C version runs %5.2f times faster" % (tp/tc))

In [ ]:
profile()